In [1]:
import re
from pathlib import Path
import os
import pandas as pd
import altair as alt
import numpy as np

In [2]:
csv_file = os.path.join(Path.cwd(), "IS 308 dataset.xlsx")
df = pd.read_excel(csv_file)
df.head()

,Name,Year,Date,Box Office,Critics Score,Audience Score,Asian Character?,Do they have a speaking role,Was asian culture depicted as being foreign/alienation,Type of Culture Represented
0,Chinatown,1974,06/20/1974,29226502,0.99,0.93,Y,Y,Y,Chinese
1,Apocalypse Now,1979,08/15/1979,104880868,0.98,0.94,N,N,Y,Vietnamese
2,Enter the Dragon,1973,08/19/1973,114252,0.88,0.91,Y,Y,N,Chinese
3,Airplane!,1980,1980-02-07 00:00:00,83454304,0.97,0.89,Y,Y,Y,Japanese
4,First Blood,1982,10/22/1982,125212904,0.86,0.86,Y,Y,Y,VIetnamese


In [3]:
def clean_yr(yr):
    yr = str(yr)
    return yr[:3]

In [4]:
df['decade'] = df["Year"].apply(clean_yr)
df['decade'] =  df['decade'].astype(str) + '0'
df['decade'].unique()
df.rename(columns = {'Asian Character?':'Test 1', 'Do they have a speaking role':'Test 2', 'Was asian culture depicted as being foreign/alienation':'Test 3'}, inplace = True)
df.head()
		

,Name,Year,Date,Box Office,Critics Score,Audience Score,Test 1,Test 2,Test 3,Type of Culture Represented,decade
0,Chinatown,1974,06/20/1974,29226502,0.99,0.93,Y,Y,Y,Chinese,1970
1,Apocalypse Now,1979,08/15/1979,104880868,0.98,0.94,N,N,Y,Vietnamese,1970
2,Enter the Dragon,1973,08/19/1973,114252,0.88,0.91,Y,Y,N,Chinese,1970
3,Airplane!,1980,1980-02-07 00:00:00,83454304,0.97,0.89,Y,Y,Y,Japanese,1980
4,First Blood,1982,10/22/1982,125212904,0.86,0.86,Y,Y,Y,VIetnamese,1980


In [5]:
def score_calc(t1, t2, t3):
    if (t3 == "N") and (t2 == "Y"):
        return 3
    elif t2 == "Y":
        return 2
    elif t1 == "Y":
        return 1
    else:
        return 0

In [6]:
source = pd.DataFrame({'Position':['Center','Center','Power Forward','Point Guard','Point Guard','Shooting Guard','Power Forward','Point Guard','Point Guard'],
                      'Points_Per_Game':[12 , 13, 15 , 21, 22, 9, 8, 5, 7],
                      'Games_Played':[22, 9, 8, 5, 7 , 12 , 13, 15 , 21]})

position_dropdown = alt.binding_select(
    options=[None] + list(source.Position.unique()), labels = ["None"] + ['All'] + list(source.Position.unique()))
position_selection = alt.selection_single(fields=['Position'], bind=position_dropdown, name='Player ')

alt.Chart(source).mark_circle(size=100).encode(
    x='Games_Played:N',
    y='Points_Per_Game:N',
).add_selection(
    position_selection,
).transform_filter(
    position_selection
)

/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [7]:
df1 = df
df1["Date"] = pd.to_datetime(df["Date"])
df1["score"] = df1.apply(lambda x: score_calc(x["Test 1"], x["Test 2"], x["Test 3"]), axis=1)
source = df1
df1.head()

,Name,Year,Date,Box Office,Critics Score,Audience Score,Test 1,Test 2,Test 3,Type of Culture Represented,decade,score
0,Chinatown,1974,1974-06-20,29226502,0.99,0.93,Y,Y,Y,Chinese,1970,2
1,Apocalypse Now,1979,1979-08-15,104880868,0.98,0.94,N,N,Y,Vietnamese,1970,0
2,Enter the Dragon,1973,1973-08-19,114252,0.88,0.91,Y,Y,N,Chinese,1970,3
3,Airplane!,1980,1980-02-07,83454304,0.97,0.89,Y,Y,Y,Japanese,1980,2
4,First Blood,1982,1982-10-22,125212904,0.86,0.86,Y,Y,Y,VIetnamese,1980,2


In [8]:
#position_dropdown = alt.binding_select(options=[None] + list(source.decade.unique()), labels = [''] + list(source.decade.unique()), name="Decade:")
selector = alt.selection_point(fields=['decade'])

color = alt.condition(
    selector,
    alt.Color('decade:N').legend(None),
    alt.value('lightgray')
)

base = alt.Chart(source).properties(
    width=500,
    height=500
)
y_d=alt.Y("decade:N", title="Decade")

bars = base.mark_bar(size=50).encode(
    x=alt.X("mean(score):Q", title="Mean of Test Score"),
    y=alt.Y("decade:N", title="Decade").sort('-x'),
    color=alt.condition(
        selector,
        y_d,
        alt.value('lightgray'),
        ),
    tooltip=[alt.Tooltip('mean(score):Q', title='Mean Test Score')],
).add_params(selector).transform_filter(selector)

yrule = (
    base.mark_rule(strokeDash=[12, 6], size=2).encode(y=alt.datum(df["score"].mean()))
)

xrule = (
    base.mark_rule(strokeDash=[12, 6], size=2).encode(x=alt.datum(df["score"].mean()))
)

lines = base.mark_line(point=True).encode(
    x='Date:T',
    y=alt.Y('score:Q', title="Score"),
    color=alt.condition(
        selector,
        y_d,
        alt.value('lightgray'),
        ),
    tooltip=[alt.Tooltip('Name:N', title='Film: '), alt.Tooltip('Date:T', title='Date released in USA'), alt.Tooltip('score:Q', title='Test Passed')],
).transform_filter(
    selector
)

dep = lines + yrule
prime = bars + xrule

viz_v1 = alt.hconcat(prime, dep).configure_point(size=120)

viz_v1.properties().save("IS308-fin-v1.json")
viz_v1


/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib

alt.HConcatChart(...)

In [9]:
def convert_t3(t):
    if (t == "Y"):
        return 1
    else:
        return 0

In [10]:
df_region = df.drop(columns=["Date"])
df_region = df_region[df_region["Test 3"] ==  "Y"]
df_region["Test 3"] = df_region["Test 3"].apply(convert_t3) 
df_region = df_region.groupby("Type of Culture Represented").agg("sum").reset_index()

df_region.head(20)

df_region[["Type of Culture Represented","Test 3"]]

,Type of Culture Represented,Test 3
0,Ambigious,1
1,Chinese,3
2,Japanese,2
3,VIetnamese,1
4,Vietnamese,1


In [11]:
base = alt.Chart(df_region).mark_arc(innerRadius=70).encode(
    theta="Test 3",
    color="Type of Culture Represented:N",
)
base

/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [12]:
x = pd.DataFrame([df.loc[15], df.loc[17], df.loc[7], df.loc[10], df.loc[20], df.loc[1]])
x["Critics Score"] = x["Critics Score"]*100
x["Audience Score"] = x["Audience Score"]*100
x["Name"].unique()
x = x.rename(columns = {"decade" : "Decade", "decade" : "Decade"})

In [13]:
input_dropdown = alt.binding_select(options=['The Dark Knight', 'Life of pi', 'Indiana Jones and the Temple of Doom', 'Mulan',
       'Everything Everywhere All at Once', 'Apocalypse Now'], name='Region ')
selection = alt.selection_point(fields=['Name'], bind=input_dropdown)


viz_v3 = alt.Chart(x).mark_bar(size=20).encode(
    x=alt.X('Percent:Q', title="Percentage Ratings"),
    y=alt.Y('Type:N', title="Type"),
    color='Type:N',
    tooltip=[alt.Tooltip('Name:N', title='Film: '), alt.Tooltip('Date:T', title='Date released in USA'), alt.Tooltip('score:Q', title='Test Passed')
    , alt.Tooltip('Audience Score:Q'), alt.Tooltip('Critics Score:Q')],
    row=alt.Row('Decade:N')
).transform_fold(
    as_=['Type', 'Percent'],
    fold=['Critics Score', 'Audience Score']
)

viz_v3.properties().save("IS308-fin-v3.json")
viz_v3


/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/nicholas/anaconda3/lib

alt.Chart(...)

In [14]:
# Was an approach I tried to use for viz 1 but this didn't pan out.
"""
selector = alt.selection_point(fields=['Name:N'], init={'Name':'First Blood'})

#color_scale = alt.Scale(domain=['1970', '1980', '1990', '2000', '2010', '2020'], range=[ '#7a5c4f', '#bb9584', '#debec9', '#b7667e', '#6e3f49', '#6d527b'])

color = alt.condition(
    selector,
    alt.Color('Name:N', scale=color_scale).legend(None),
    alt.value('lightgray')
)

base = alt.Chart(source).properties(
    width=500,
    height=500
).add_params(selector).transform_filter(
    selector
)

bars = base.mark_circle(size=120).encode(
    x=alt.X("score:Q"),
    y=alt.Y("Box Office:Q"),
    color=alt.condition(
        selector,
        'Name:N',
        alt.value('lightgray'),
        ),
    tooltip=[alt.Tooltip('mean(score):Q', title='Average Bechdel Test Score'), alt.Tooltip('count(decade):Q', title='Number of films in franchise')],
).properties()

lines = alt.Chart(df).mark_bar().encode(
    x='Percent:Q',
    y='type:N',
    color='type:N',
    row=alt.condition(
        selector,
        'Name:N',
        ),
).transform_fold(
    as_=['type', 'Percent'],
    fold=['Critics Score', 'Audience Score']
).add_params(
    selector
).transform_filter(
    selector
)

dep = lines #+ yrule
prime = bars #+ xrule

viz_v1 = alt.hconcat(prime, dep).configure_point(size=120)

viz_v1.properties().save("IS308-fin-v1.json")
viz_v1
"""

/home/nicholas/anaconda3/lib/python3.11/site-packages/altair/vegalite/v5/api.py:381: AltairDeprecationWarning: Use 'value' instead of 'init'.
  warnings.warn(


TypeError: altair.vegalite.v5.schema.core.SelectionParameter() got multiple values for keyword argument 'value'